In [1]:
from scipy.io import loadmat
from scipy.stats import zscore
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib

import plotly.offline as pyo
import plotly.graph_objects as go
from plotly import subplots

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

Using matplotlib backend: Qt5Agg


In [28]:
data = loadmat('firingvector-20130711')
QNREM = data['spks_train']
region = data['region']
type = data['type']
issws = data['issws']
isrem = data['isrem']
for i,r in enumerate(region):
    region[i] = r.strip()


for i,r in enumerate(type):
    type[i] = r.strip()
    
QNREM[QNREM>1] = 1
QALL = [QNREM.T[region == i] for i in np.unique(region)]

In [4]:
def FiringFeatures(Q):
    state = []
    movmean = np.array(np.empty([np.shape(Q)[0],1]))
    for i in range(0,np.shape(Q)[1],20):
        movmean = np.hstack((movmean,np.sum(Q[:,i:i+200],1).reshape(-1,1)))
        if issws[i]: state.append('sws')
        if isrem[i]:state.append('rem')
    movmean = np.delete(movmean,0,1)

    c = np.corrcoef(movmean.T)
    
    return c

In [5]:

state = []
for i in range(0,np.shape(QNREM)[0]):
    if issws[i]: state.append('sws')
    if isrem[i]: state.append('rem')

4001


In [27]:
app = dash.Dash()

app.layout = html.Div([
    dcc.Graph(id = 'FiringFeatures'),
    dcc.RangeSlider(id = 'interval',min = 0, max = np.shape(QNREM)[0],value = [0 ,1]),
    html.Div(dcc.Slider(id = 'zmin',min = 0, max = 1, value = 0.1,step = 0.1))
    ],style={'width':'80%','position':'centered'})


@app.callback(Output('FiringFeatures','figure'),
             [Input('interval','value'),
              Input('zmin','value')])


def updateDash(interval,zmin):
    t1 = interval[0]
    t2 = interval[1]

    data = [go.Heatmap(z = FiringFeatures(QALL[i][:,t1:t2]),
                    zmin = zmin,
                    zmax = 1,
                    showscale = True,
                    colorscale = 'jet') for i in range(len(QALL))]

    state_go = [go.Scatter(y = state[t1:t2],
                        mode = 'lines',
                        marker = {'color':'red'}) for i in np.unique(region)]

    layout = go.Layout(title = 'Firing Features Correlations',
                    xaxis = {'title':'test'},
                    height=700)

    fig = subplots.make_subplots(2,len(QALL),
                                subplot_titles=[i for i in np.unique(region)],
                                shared_yaxes = True,
                                shared_xaxes = False,
                                vertical_spacing = 0.05
                             )
                

    for i in range(len(QALL)):
        fig.append_trace(data[i],1,i+1)
        fig.append_trace(state_go[i],2,i+1)

    fig['layout'].update(layout)

    fig.update_yaxes(autorange="reversed")

    return fig



app.run_server()

 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/home/billel/anaconda3/envs/plotly/lib/python3.8/site-packages/dash/dash.py", line 1371, in dispatch
    response.set_data(self.callback_map[output]["callback"](*args))
  File "/home/billel/anaconda3/envs/plotly/lib/python3.8/site-packages/dash/dash.py", line 1251, in add_context
    output_value = func(*args, **kwargs)  # %% callback invoked %%
  File "<ipython-input-27-a5188dde1c71>", line 19, in updateDash
    data = [go.Heatmap(z = FiringFeatures(QALL[i][:,t1:t2]),
  File "<ipython-input-27-a5188dde1c71>", line 19, in <listcomp>
    data = [go.Heatmap(z = FiringFeatures(QALL[i][:,t1:t2]),
  File "/home/billel/anaconda3/envs/plotly/lib/python3.8/site-packages/plotly/graph_objs/__init__.py", line 66362, in __init__
    self["z"] = z if z is not None else _v
  File "/home/billel/anaconda3/envs/plotly/lib/python3.8/site-packages/plotly/basedatatypes.py", line 3490, in __setitem__
    self.

In [12]:
QALL[0][:,5]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0], dtype=uint8)

In [21]:
np.size(state)

11

In [21]:
help(dcc.Slider)

Help on class Slider in module dash_core_components.Slider:

class Slider(dash.development.base_component.Component)
 |  Slider(id=undefined, marks=undefined, value=undefined, className=undefined, disabled=undefined, dots=undefined, included=undefined, min=undefined, max=undefined, tooltip=undefined, step=undefined, vertical=undefined, updatemode=undefined, loading_state=undefined, persistence=undefined, persisted_props=undefined, persistence_type=undefined, **kwargs)
 |  
 |  A Slider component.
 |  A slider component with a single handle.
 |  
 |  Keyword arguments:
 |  - id (string; optional): The ID of this component, used to identify dash components
 |  in callbacks. The ID needs to be unique across all of the
 |  components in an app.
 |  - marks (dict; optional): Marks on the slider.
 |  The key determines the position (a number),
 |  and the value determines what will show.
 |  If you want to set the style of a specific mark point,
 |  the value should be an object which
 |  co